In [ ]:
# "Implicit Prop.py"
#SEG TREE (Implicit Prop)
class SegmentTree:
    def __init__(self, input_arr):
        n = 1<<(len(input_arr).bit_length()-1)
        if n < len(input_arr): n<<=1
        self.n = n
        self.arr = [0]*n + input_arr + [0]*(n-len(input_arr))
        self.pend = [0]*(n*2)
        for i in range(n-1,0,-1):
            self.arr[i] = self.arr[i*2]+self.arr[i*2+1]
            
    def flush_pending(self):
        n = self.n
        for i in range(n):
            self.pend[i*2]+=self.pend[i]
            self.pend[i*2+1]+=self.pend[i]
        for i in range(n, n*2):
            self.arr[i]+=self.pend[i]
        self.pend = [0]*(n*2)
        for i in range(n-1, 0, -1):
            self.arr[i] = self.arr[i*2]+self.arr[i*2+1]
            
    __list__ = lambda self: self.flush_pending() or self.arr[-self.n:]
    __str__ = lambda self: str(self.__list__())
    
    def update(self, l, r, x):
        st = [(1, l, r, self.n)]
        while st:
            u, l, r, n = st.pop()
            l = max(0, l)
            r = min(n, r)
            if r-l==n:
                self.pend[u]+=x
                continue
            else:
                self.arr[u]+=(x*(r-l))
            n//=2
            if l<n:
                st.append((u*2, l, r, n))
            if r>n:
                st.append((u*2+1, l-n, r-n, n))
            
        
    def query(self, l, r):
        res = 0
        st = [(1, l, r, self.n)]
        while st:
            u, l, r, n = st.pop()
            self.arr[u]+=self.pend[u]*n
            if n != 1:
                self.pend[u*2]+=self.pend[u]
                self.pend[u*2+1]+=self.pend[u]
            self.pend[u]=0
            l = max(0, l)
            r = min(n, r)
            if r-l==n:
                res+=self.arr[u]
                continue
            n//=2
            if l<n:
                st.append((u*2, l, r, n))
            if r>n:
                st.append((u*2+1, l-n, r-n, n))
        return res

if __name__ == '__main__':
    import random
    import time
    testcases, operations, m = 10, 10, 1000000
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(-m, m), random.randint(m, m+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l_update, r_update = l, r
            x = random.randint(-5, 5)
            tree_op_start = time.time()
            tree.update(l, r, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            for i in range(l, r):
                arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = sum(arr[l:r])
            naive_time+=time.time()-naive_query_start
            if tree_val != arr_val:
                # print(tree.arr)
                # print(tree.pend)
                print(tree)
                print(arr)
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(l_update, r_update, x)}, query: {(l, r)}')
        #     print(f'operation {op} passed!')
        # print(f'testcase {t} passed!')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')

In [ ]:
# "Implicit Prop Old.py"
#SEG TREE (Implicit Prop)
        
class SegmentTree:
    def __init__(self, input_arr):
        self.n = n = len(input_arr)
        self.arr = arr = [None]*(n*2)
        self.pop = pop = [0]*(n*2)
        self.acc = acc = [0]*(n*2)
        self.pend = pend = [0]*(n*2)
        st = [(0, n, 1, 0)]
        while st:
            l, r, d, i = st.pop()
            if l == r: continue
            if not d:
                self._refresh(i)
                continue
            else:
                m = l+(r-l)//2
                arr[i] = input_arr[m]
                st.append((l, r, 0, i))
                st.append((m+1, r, 1, i*2+2))
                st.append((l, m, 1, i*2+1))
    
    def __str__(self):
        arr = self.arr
        st = [(0, 1)]
        res = []
        while st:
            i, d = st.pop()
            self._propagate(i)
            if d:
                if i*2+2<self.n*2 and self.arr[i*2+2] is not None: st.append((i*2+2, 1))
                st.append((i, 0))
                if i*2+1<self.n*2 and self.arr[i*2+1] is not None: st.append((i*2+1, 1))
            else:
                res.append((arr[i]))
        return str(res)
                
    def update(self, l, r, val):
        arr, pop, acc, pend = self.arr, self.pop, self.acc, self.pend
        st = [(0, l, r, 1)]
        while st:
            i, l, r, d = st.pop()
            if not d:
                self._refresh(i)
                continue
            l = max(l, 0)
            r = min(r, pop[i])
            if not l and r == pop[i]:
                pend[i]+=val
                continue
            l_pop = self._count(i*2+1)
            if l<=l_pop and r>l_pop:
                arr[i]+=val
            st.append((i, l, r, 0))
            if r > l_pop+1:
                st.append((i*2+2, l-l_pop-1, r-l_pop-1, 1))
            if l < l_pop:
                st.append((i*2+1, l, r, 1))
            
    
    def query(self, l, r):
        arr, pop, acc, pend = self.arr, self.pop, self.acc, self.pend
        res = 0 #Identity element
        st = [(0, l, r)]
        while st:
            i, l, r = st.pop()
            self._propagate(i)
            l = max(l, 0)
            r = min(r, pop[i])
            if not l and r == pop[i]:
                res += acc[i]
                continue
            l_pop = self._count(i*2+1)
            res += arr[i]*(l<=l_pop)*(r>l_pop)
            if r > l_pop+1:
                st.append((i*2+2, l-l_pop-1, r-l_pop-1))
            if l < l_pop:
                st.append((i*2+1, l, r))
        return res

    
    def _refresh(self, i):
        self._propagate(i)
        self.acc[i] = self._aggr(i*2+1)+self.arr[i]+self._aggr(i*2+2)
        self.pop[i] = self._count(i*2+1)+1+self._count(i*2+2)
        
    def _propagate(self, i):
        self.arr[i] += self.pend[i]
        self.acc[i] += self.pend[i]*self.pop[i]
        if i*2+1<self.n*2 and self.arr[i*2+1] is not None: self.pend[i*2+1]+=self.pend[i]
        if i*2+2<self.n*2 and self.arr[i*2+2] is not None: self.pend[i*2+2]+=self.pend[i]
        self.pend[i] = 0 #Identity element
        
    def _aggr(self, i):
        if i<self.n*2 and self.arr[i] is not None:
            self._propagate(i)
            return self.acc[i]
        else:
            return 0
    
    def _count(self, i):
        return self.pop[i] if i<self.n*2 and self.arr[i] is not None else 0

    
    

if __name__ == '__main__':
    import random
    import time
    testcases, operations, n = 100, 100, 100
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(-n, n), random.randint(n, n+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l_update, r_update = l, r
            x = random.randint(-5, 5)
            tree_op_start = time.time()
            tree.update(l, r, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            for i in range(l, r):
                arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = sum(arr[l:r])
            naive_time+=time.time()-naive_query_start
            if tree_val != arr_val:
                print(tree)
                print(arr)
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(l_update, r_update, x)}, query: {(l, r)}')
        #     print(f'operation {op} passed!')
        # print(f'testcase {t} passed!')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')

In [ ]:
# "Implicit Prop Max.py"
#SEG TREE (Implicit Prop, Max Query)
from math import inf
class SegmentTree:
    def __init__(self, input_arr):
        n = 1<<(len(input_arr).bit_length()-1)
        if n < len(input_arr): n<<=1
        self.n = n
        self.arr = [-inf]*n + input_arr + [-inf]*(n-len(input_arr))
        self.pend = [0]*(n*2)
        for i in range(n-1,0,-1):
            self.arr[i] = max(self.arr[i*2], self.arr[i*2+1])
            
    def flush_pending(self):
        n = self.n
        for i in range(n):
            self.pend[i*2]+=self.pend[i]
            self.pend[i*2+1]+=self.pend[i]
        for i in range(n, n*2):
            self.arr[i]+=self.pend[i]
        self.pend = [0]*(n*2)
        for i in range(n-1, 0, -1):
            self.arr[i] = max(self.arr[i*2], self.arr[i*2+1])
            
    __list__ = lambda self: self.flush_pending() or self.arr[-self.n:]
    __str__ = lambda self: str(self.__list__())
    
    def update(self, l, r, x):
        st = [(1, l, r, self.n, 1)]
        while st:
            u, l, r, n, d = st.pop()
            l = max(0, l)
            r = min(n, r)
            if r-l==n:
                self.pend[u]+=x
                continue
            if not d:
                self.arr[u] = max(self.arr[u*2]+self.pend[u*2], self.arr[u*2+1]+self.pend[u*2+1])
                continue
            st.append((u, l, r, n, 0))
            n//=2
            if l<n:
                st.append((u*2, l, r, n, 1))
            if r>n:
                st.append((u*2+1, l-n, r-n, n, 1))
            
        
    def query(self, l, r):
        res = -inf
        st = [(1, l, r, self.n)]
        while st:
            u, l, r, n = st.pop()
            self.arr[u]+=self.pend[u]
            if n != 1:
                self.pend[u*2]+=self.pend[u]
                self.pend[u*2+1]+=self.pend[u]
            self.pend[u]=0
            l = max(0, l)
            r = min(n, r)
            if r-l==n:
                res= max(res, self.arr[u])
                continue
            n//=2
            if l<n:
                st.append((u*2, l, r, n))
            if r>n:
                st.append((u*2+1, l-n, r-n, n))
        return res

if __name__ == '__main__':
    import random
    import time
    testcases, operations, m = 10, 10, 100000
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(-m, m), random.randint(m, m+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l_update, r_update = l, r
            x = random.randint(-5, 5)
            tree_op_start = time.time()
            tree.update(l, r, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            for i in range(l, r):
                arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = max(arr[l:r])
            naive_time+=time.time()-naive_query_start
            if tree_val != arr_val:
                # print(tree.arr)
                # print(tree.pend)
                print(tree)
                print(arr)
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(l_update, r_update, x)}, query: {(l, r)}')
        #     print(f'operation {op} passed!')
        # print(f'testcase {t} passed!')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')


In [ ]:
# "Implicit No Prop.py"
#SEG TREE (Implicit No Prop)
        
class SegmentTree:
    def __init__(self, input_arr):
        n = 1<<(len(input_arr).bit_length()-1)
        if n < len(input_arr): n<<=1
        self.n = n
        self.arr = arr =[0]*n + input_arr + [0]*(n-len(input_arr))
        for i in range(n-1,0,-1):
            arr[i] = arr[i*2]+arr[i*2+1]
    
    def __list__(self):
        return self.arr
    def __str__(self):
        return str(self.__list__())
    def update(self, i, val):
        u, bit = 1, self.n>>1
        while bit:
            self.arr[u]+=val
            u*=2
            u+=bool(bit&i)
            bit>>=1
        self.arr[u]+=val
            
            
    def query(self, l, r):
        res = 0
        st = [(1, l, r, self.n)]
        while st:
            u, l, r, n = st.pop()
            l = max(l, 0)
            r = min(r, n)
            if r-l==n:
                res+=self.arr[u]
                continue
            n//=2
            if l<n:
                st.append((u*2, l, r, n))
            if r>n:
                st.append((u*2+1, l-n, r-n, n))
        return res
    

        
if __name__ == '__main__':
    import random
    import time
    testcases, operations, n = 10, 100, 300
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(0, n), n))#random.randint(n, n+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            i = random.randint(0, n-1)
            i_update = i
            x = random.randint(-5, 5)
            tree_op_start = time.time()
            tree.update(i, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l, r = 0, n
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = sum(arr[l:r])
            naive_time+=time.time()-naive_query_start
            if tree_val != arr_val:
                print(tree)
                print(arr)
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(i_update, x)}, query: {(l, r)}')
        #     print(f'operation {op} passed!')
        # print(f'testcase {t} passed!')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')

In [ ]:
# "Implicit No Prop Old.py"
#SEG TREE (Implicit No Prop)
        
class SegmentTree:
    def __init__(self, input_arr):
        n = 1<<(len(input_arr).bit_length()-1)
        if n < len(input_arr): n<<=1
        self.n = n
        self.arr = arr =[-inf]*n + input_arr + [-inf]*(n-len(input_arr))
        for i in range(n-1,0,-1):
            arr[i] = max(arr[i*2], arr[i*2+1])
    
    def __list__(self):
        return self.arr
    def __str__(self):
        return str(self.__list__())
    def update(self, i, val):
        st = []
        u, bit = 1, self.n>>1
        while bit:
            st.append(u)
            u*=2
            u+=bool(bit&i)
            bit>>=1
        self.arr[u] = val
        while st:
            u = st.pop()
            self.arr[u]=max(self.arr[u*2], self.arr[u*2+1])
            
            
    def query(self, l, r):
        res = -inf
        st = [(1, l, r, self.n)]
        while st:
            u, l, r, n = st.pop()
            l = max(l, 0)
            r = min(r, n)
            if r-l==n:
                res = max(res, self.arr[u])
                continue
            n//=2
            if l<n:
                st.append((u*2, l, r, n))
            if r>n:
                st.append((u*2+1, l-n, r-n, n))
        return res
    
    

if __name__ == '__main__':
    import random
    import time
    testcases, operations, n = 1, 50, 1000000
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(-n, n), random.randint(n, n+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            i = random.randint(0, n-1)
            i_update = i
            x = random.randint(-5, 5)
            tree_op_start = time.time()
            tree.update(i, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l, r = 0, n
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = sum(arr[l:r])
            naive_time+=time.time()-naive_query_start
            if tree_val != arr_val:
                print(tree)
                print(arr)
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(i_update, x)}, query: {(l, r)}')
        #     print(f'operation {op} passed!')
        # print(f'testcase {t} passed!')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')

In [ ]:
# "Implicit No Prop Max.py"
#SEG TREE (Implicit No Prop)
from math import inf
class SegmentTree:
    def __init__(self, input_arr):
        n = 1<<(len(input_arr).bit_length()-1)
        if n < len(input_arr): n<<=1
        self.n = n
        self.arr = arr =[-inf]*n + input_arr + [-inf]*(n-len(input_arr))
        for i in range(n-1,0,-1):
            arr[i] = max(arr[i*2], arr[i*2+1])
    
    def __list__(self):
        return self.arr
    def __str__(self):
        return str(self.__list__())
    def update(self, i, val):
        arr = self.arr
        u, bit = 1, self.n>>1
        st = [(u, bit, 1)]
        while st:
            u, bit, d = st.pop()
            if not bit:
                arr[u]+=val
                continue
            if not d:
                arr[u] = max(arr[u*2], arr[u*2+1])
                continue
            st.append((u, bit, 0))
            self.arr[u]+=val
            u*=2
            u+=bool(bit&i)
            bit>>=1
            st.append((u, bit, 1))
            
            
    def query(self, l, r):
        res = -inf
        st = [(1, l, r, self.n)]
        while st:
            u, l, r, n = st.pop()
            l = max(l, 0)
            r = min(r, n)
            if r-l==n:
                res=max(res, self.arr[u])
                continue
            n//=2
            if l<n:
                st.append((u*2, l, r, n))
            if r>n:
                st.append((u*2+1, l-n, r-n, n))
        return res
    
            
if __name__ == '__main__':
    import random
    import time
    testcases, operations, n = 1, 20, 1000000
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        # arr = list(random.sample(range(-n, n), random.randint(n, n+5)))
        arr = [0]*n
        naive_time+=time.time()-naive_start
        tree_start = time.time()
        tree = SegmentTree([0]*n)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l_update, r_update = l, r
            x = random.randint(-5, 5)
            tree_op_start = time.time()
            tree.update(l, r, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            for i in range(l, r):
                arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l, r = 0, n
            tree_query_start = time.time()
            tree_val = tree[l:r]
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = max(arr[l:r])
            naive_time+=time.time()-naive_query_start
            if tree_val != arr_val:
                print(tree)
                print(arr)
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(l_update, r_update, x)}, query: {(l, r)}')
        #     print(f'operation {op} passed!')
        # print(f'testcase {t} passed!')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')

In [ ]:
# "Explicit Prop.py"
#SEG TREE (Prop)
class SegmentNode:
    def __init__(self, v=0):
        self.pop = 1
        self.pend = 0
        self.val = self.sum = v
        self.left = self.right = None
        
class SegmentTree:
    def __init__(self, arr):
        n = len(arr)
        node_arr = [None]*n
        st = [(0, n, None)]
        while st:
            l, r, node = st.pop()
            if l == r: continue
            if node:
                m = l+(r-l)//2
                if m>l: node.left = node_arr[l+(m-l)//2]
                if r>m+1: node.right = node_arr[m+1+(r-m-1)//2]
                self._refresh(node)
                continue
            else:
                m = l+(r-l)//2
                node = SegmentNode(arr[m])
                node_arr[m]=node
                st.append((l, r, node))
                st.append((m+1, r, None))
                st.append((l, m, None))
        self.root = node
    
    def __str__(self):
        st = [(self.root, 1)]
        res = []
        while st:
            node, d = st.pop()
            self._propagate(node)
            if d:
                if u.right: st.append((node.right, 1))
                st.append((node, 0))
                if u.left: st.append((node.left, 1))
            else:
                res.append((node.val))
        return str(res)
                
    def update(self, l, r, val):
        st = [(self.root, l, r, 1)]
        while st:
            node, l, r, d = st.pop()
            if not d:
                self._refresh(node)
                continue
            l = max(l, 0)
            r = min(r, node.pop)
            if not l and r == node.pop:
                node.pend+=val
                continue
            l_pop = self._count(node.left)
            if l<=l_pop and r>l_pop:
                node.val+=val
            st.append((node, l, r, 0))
            if r > l_pop+1:
                st.append((node.right, l-l_pop-1, r-l_pop-1, 1))
            if l < l_pop:
                st.append((node.left, l, r, 1))
            
    
    def query(self, l, r):
        res = 0 #Identity element
        st = [(self.root, l, r)]
        while st:
            node, l, r = st.pop()
            self._propagate(node)
            l = max(l, 0)
            r = min(r, node.pop)
            if not l and r == node.pop:
                res += node.sum
                continue
            l_pop = self._count(node.left)
            res += node.val*(l<=l_pop)*(r>l_pop)
            if r > l_pop+1:
                st.append((node.right, l-l_pop-1, r-l_pop-1))
            if l < l_pop:
                st.append((node.left, l, r))
        return res

    
    def _refresh(self, node):
        self._propagate(node)
        node.sum = self._aggr(node.left)+node.val+self._aggr(node.right)
        node.pop = self._count(node.left)+1+self._count(node.right)
        
    def _propagate(self, node):
        node.val += node.pend
        node.sum += node.pend*node.pop
        if node.left: node.left.pend+=node.pend
        if node.right: node.right.pend+=node.pend
        node.pend = 0
        
    def _aggr(self, node):
        if node: self._propagate(node)
        return node.sum if node else 0
    
    def _count(self, node):
        return node.pop if node else 0

    
    

if __name__ == '__main__':
    import random
    import time
    testcases, operations, n = 100, 100, 5000
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(-n, n), random.randint(n, n+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            x = random.randint(-100, 100)
            tree_op_start = time.time()
            tree.update(l, r, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            for i in range(l, r):
                arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = sum(arr[l:r])
            if tree_val != arr_val:
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(i, x)}, query: {(l, r)}')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')

In [ ]:
# "Explicit Prop Add-Mul.py"
#SEG TREE (Prop Mul + Add)
mod = int(1e9+7)
class SegmentNode:
    def __init__(self, v=0):
        self.pop = 1
        self.pend_add = 0
        self.pend_mul = 1
        self.val = self.sum = v
        self.left = self.right = None
        
class SegmentTree:
    def __init__(self, arr):
        n = len(arr)
        node_arr = [None]*n
        st = [(0, n, None)]
        while st:
            l, r, node = st.pop()
            if l == r: continue
            if node:
                m = l+(r-l)//2
                if m>l: node.left = node_arr[l+(m-l)//2]
                if r>m+1: node.right = node_arr[m+1+(r-m-1)//2]
                self._refresh(node)
                continue
            else:
                m = l+(r-l)//2
                node = SegmentNode(arr[m])
                node_arr[m]=node
                st.append((l, r, node))
                st.append((m+1, r, None))
                st.append((l, m, None))
        self.root = node
    
    def __str__(self):
        st = [(self.root, 1)]
        res = []
        while st:
            node, d = st.pop()
            self._propagate(node)
            if d:
                if node.right: st.append((node.right, 1))
                st.append((node, 0))
                if node.left: st.append((node.left, 1))
            else:
                res.append((node.val))
        return str(res)
                
    #o: 0 is add, 1 is mul
    def update(self, l, r, val, o):
        st = [(self.root, l, r, 1)]
        while st:
            node, l, r, d = st.pop()
            if not d:
                self._refresh(node)
                continue
            self._propagate(node)
            l = max(l, 0)
            r = min(r, node.pop)
            if not l and r == node.pop:
                if o:
                    node.pend_mul*=val
                    node.pend_add*=val
                else:
                    node.pend_add+=val
                node.pend_mul%=mod
                node.pend_add%=mod
                continue
            l_pop = self._count(node.left)
            if l<=l_pop and r>l_pop:
                if o:
                    node.val*=val
                else:
                    node.val+=val
                node.val%=mod
            st.append((node, l, r, 0))
            if r > l_pop+1:
                st.append((node.right, l-l_pop-1, r-l_pop-1, 1))
            if l < l_pop:
                st.append((node.left, l, r, 1))
            
    
    def query(self, l, r):
        res = 0 #Identity element
        st = [(self.root, l, r)]
        while st:
            node, l, r = st.pop()
            self._propagate(node)
            l = max(l, 0)
            r = min(r, node.pop)
            if not l and r == node.pop:
                res += node.sum
                continue
            l_pop = self._count(node.left)
            res += node.val*(l<=l_pop)*(r>l_pop)
            if r > l_pop+1:
                st.append((node.right, l-l_pop-1, r-l_pop-1))
            if l < l_pop:
                st.append((node.left, l, r))
        return res%mod

    
    def _refresh(self, node):
        self._propagate(node)
        node.sum = (self._aggr(node.left)+node.val+self._aggr(node.right))%mod
        node.pop = self._count(node.left)+1+self._count(node.right)
        
    def _propagate(self, node):
        node.val *= node.pend_mul
        node.sum *= node.pend_mul
        node.val += node.pend_add
        node.sum += node.pend_add*node.pop
        node.val%=mod
        node.sum%=mod
        if node.left:
            node.left.pend_mul*=node.pend_mul
            node.left.pend_add*=node.pend_mul
            node.left.pend_add+=node.pend_add
            node.left.pend_mul%=mod
            node.left.pend_add%=mod
        if node.right:
            node.right.pend_mul*=node.pend_mul
            node.right.pend_add*=node.pend_mul
            node.right.pend_add+=node.pend_add
            node.right.pend_mul%=mod
            node.right.pend_add%=mod
        node.pend_mul = 1
        node.pend_add = 0
        
    def _aggr(self, node):
        if node: self._propagate(node)
        return node.sum if node else 0
    
    def _count(self, node):
        return node.pop if node else 0

    
    

if __name__ == '__main__':
    import random
    import time
    testcases, operations, n = 1, 100, 100000
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(-n, n), random.randint(n, n+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            l_update, r_update = l, r
            x = random.randint(1, 100)
            o = random.randint(0, 1)
            tree_op_start = time.time()
            tree.update(l, r, x, o)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            if o:
                for i in range(l, r):
                    arr[i]*=x
                    arr[i]%=mod
            else:
                for i in range(l, r):
                    arr[i]+=x
                    arr[i]%=mod
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = sum(arr[l:r])%mod
            if tree_val != arr_val:
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(l_update, r_update, x, o)}, query: {(l, r)}')
            # if not t:
        #         print(f'operation {op} passed! (type {o})')
        # print(f'testcase {t} passed! (type {o})')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')

In [ ]:
# "Explicit No Prop.py"
#SEG TREE (No Prop)
class SegmentNode:
    def __init__(self, v=0):
        self.pop = 1
        self.val = self.sum = v
        self.left = self.right = None
        
class SegmentTree:
    def __init__(self, arr):
        n = len(arr)
        node_arr = [None]*n
        st = [(0, n, None)]
        while st:
            l, r, node = st.pop()
            if l == r: continue
            if node:
                m = l+(r-l)//2
                if m>l: node.left = node_arr[l+(m-l)//2]
                if r>m+1: node.right = node_arr[m+1+(r-m-1)//2]
                self._refresh(node)
                continue
            else:
                m = l+(r-l)//2
                node = SegmentNode(arr[m])
                node_arr[m]=node
                st.append((l, r, node))
                st.append((m+1, r, None))
                st.append((l, m, None))
        self.root = node
    
    def __str__(self):
        st = [(self.root, 1)]
        res = []
        while st:
            u, d = st.pop()
            if d:
                if u.right: st.append((u.right, 1))
                st.append((u, 0))
                if u.left: st.append((u.left, 1))
            else:
                res.append((u.val))
        return str(res)
                
    def update(self, idx, val):
        st = [self.root]
        while 1:
            node = st[-1]
            l_pop = self._count(node.left)
            if l_pop>idx:
                st.append(node.left)
            elif l_pop<idx:
                idx-=l_pop+1
                st.append(node.right)
            else:
                break
        node.val += val
        while st:
            self._refresh(st.pop())
    
    def query(self, l, r):
        # return self.query_helper(self.root, l, r)
        res = 0 #Identity element
        st = [(self.root, l, r)]
        while st:
            node, l, r = st.pop()
            l = max(l, 0)
            r = min(r, node.pop)
            if not l and r == node.pop:
                res += node.sum
                continue
            l_pop = self._count(node.left)
            res += node.val*(l<=l_pop)*(r>l_pop)
            if r > l_pop+1:
                st.append((node.right, l-l_pop-1, r-l_pop-1))
            if l < l_pop:
                st.append((node.left, l, r))
        return res

    def _refresh(self, node):
        #propagate
        node.sum = self._aggr(node.left)+node.val+self._aggr(node.right)
        node.pop = self._count(node.left)+1+self._count(node.right)
        
    def _aggr(self, node):
        return node.sum if node else 0
    
    def _count(self, node):
        return node.pop if node else 0
    
    
    

if __name__ == '__main__':
    import random
    import time
    testcases, operations, n = 100, 100, 1000
    tree_time = 0
    naive_time = 0
    for t in range(testcases):
        naive_start = time.time()
        arr = list(random.sample(range(-n, n), random.randint(5, n+5)))
        naive_time+=time.time()-naive_start
        n = len(arr)
        tree_start = time.time()
        tree = SegmentTree(arr)
        tree_time+=time.time()-tree_start
        for op in range(operations):
            i, x = random.randint(0, len(arr)-1), random.randint(-100, 100)
            tree_op_start = time.time()
            tree.update(i, x)
            tree_time += time.time() - tree_op_start
            naive_op_start = time.time()
            arr[i]+=x
            naive_time += time.time() - naive_op_start
            l = random.randint(0, n-1)
            r = random.randint(l+1, n)
            tree_query_start = time.time()
            tree_val = tree.query(l, r)
            tree_time += time.time() - tree_query_start
            naive_query_start = time.time()
            arr_val = sum(arr[l:r])
            if tree_val != arr_val:
                raise ValueError(f'naive and tree values are different. values: {(tree_val, arr_val)},  update: {(i, x)}, query: {(l, r)}')
            
            
    print(f'all {testcases} test cases passed!')
    print(f'{testcases*operations} total operations')
    print(f'Naive operation time: {naive_time}')
    print(f'Segment Tree operation time: {tree_time}')